In [1]:
import os
import re
import glob
from datetime import datetime
import sys
sys.path.append("..") # Adds higher directory to python modules path for importing from src dir

import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from tqdm import tqdm_notebook as tqdm

import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

%matplotlib inline
%load_ext autotime
%load_ext autoreload
%autoreload 2

c:\users\thomas\hpi\ma\venv\lib\importlib\_bootstrap.py:205: RuntimeWarning: cymem.cymem.Pool size changed, may indicate binary incompatibility. Expected 48 from C header, got 64 from PyObject
  return f(*args, **kwds)
c:\users\thomas\hpi\ma\venv\lib\importlib\_bootstrap.py:205: RuntimeWarning: cymem.cymem.Address size changed, may indicate binary incompatibility. Expected 24 from C header, got 40 from PyObject
  return f(*args, **kwds)
c:\users\thomas\hpi\ma\venv\lib\importlib\_bootstrap.py:205: RuntimeWarning: cymem.cymem.Pool size changed, may indicate binary incompatibility. Expected 48 from C header, got 64 from PyObject
  return f(*args, **kwds)
c:\users\thomas\hpi\ma\venv\lib\importlib\_bootstrap.py:205: RuntimeWarning: cymem.cymem.Address size changed, may indicate binary incompatibility. Expected 24 from C header, got 40 from PyObject
  return f(*args, **kwds)


In [2]:
HOME = ".."
DATA_DIR = "data"
REUTERS = os.path.join(HOME, DATA_DIR, "preprocessed", "news_reuters.csv")
BLOOMBERG = os.path.join(HOME, DATA_DIR, "preprocessed", "news_bloomberg.csv")
NEWS = os.path.join(HOME, DATA_DIR, "preprocessed", "news.csv")
# Columns: 'date', 'filename', 'content'

time: 122 ms


### Get companies

In [64]:
from src.datasets import NyseSecuritiesDataset
from src.datasets import NyseStocksDataset
import src.nlp_utils as nlp_utils

time: 805 ms


In [4]:
securities_ds = NyseSecuritiesDataset(file_path='../data/nyse/securities.csv')
companies = securities_ds.get_all_company_names()  # List[Tuple[symbol, name]]

time: 136 ms


### Apply on Reuters Article

In [5]:
# idxmax reuters: 61727  (FB 26, MSFT 1, NWSA 44, NWS 44, YHOO 1)
reuters = pd.read_csv(REUTERS, skiprows=61727, nrows=1, index_col=0)
print(reuters.loc[61727][2])

-- Special report: How News Corp got lost in Myspace
-- By  Yinka Adegoke
-- Thu Apr 7, 2011 1:08pm EDT
-- http://www.reuters.com/article/2011/04/07/us-myspace-idUSTRE7364G420110407

 

 NEW YORK  (Reuters) - As Rupert Murdoch stepped into the Grand Ballroom of San Francisco's Palace Hotel at the Web 2.0 conference in October 2007, the developers and engineers who packed the room fell into a respectful hush. It was the sort of greeting usually reserved for one of their own tech heroes, not a 76-year-old media mogul. 

 Sitting cross-legged on a red leather couch, Murdoch looked relaxed in an open-necked shirt. Next to him sat Chris De Wolfe, the co-founder of Myspace, the social network that News Corp's chairman had acquired for $580 million two years earlier. De Wolfe had spent weeks trying to sell Murdoch's advisers on the conference, which he saw as a kind of digital coming out party for his boss. But those close to the News Corp chairman worried he might embarrass himself in front 

time: 612 ms


In [61]:
article1 = nlp(reuters.loc[61727][2])
labels = [x.label_ for x in article1.ents]
print(Counter(labels))
items = [x.text for x in article1.ents if x.label_ == 'ORG']
print(Counter(items))  # .most_common(3)
sentences = [x for x in article1.sents]
print(sentences[20])
displacy.render(nlp(str(sentences[20])), jupyter=True, style='ent')

Counter({'ORG': 234, 'PERSON': 117, 'DATE': 76, 'CARDINAL': 33, 'GPE': 26, 'MONEY': 15, 'ORDINAL': 13, 'PERCENT': 7, 'FAC': 4, 'LOC': 3, 'NORP': 2, 'TIME': 1, 'PRODUCT': 1, 'WORK_OF_ART': 1})
Counter({'Myspace': 100, 'News Corp': 34, 'Facebook': 24, "News Corp's": 8, 'Google': 5, 'Viacom': 4, 'Reuters': 3, 'Allen & Co': 2, 'Sony Music': 2, 'Yahoo': 2, 'Microsoft': 2, 'AOL': 2, 'Miller': 2, 'Museum of Modern Art': 1, 'Austin Ventures': 1, 'Universal Music': 1, 'Abu Dhabi Holdings': 1, 'FAILURES Plenty': 1, 'AOL-Time Warner': 1, 'Delphi': 1, 'Fox Interactive Media': 1, 'MarketWatch': 1, 'Career Search': 1, 'IGN': 1, 'CNET': 1, 'Intermix Media': 1, 'Redpoint Ventures': 1, 'Intermix': 1, 'Redstone': 1, 'Freston': 1, 'News Corp.': 1, 'FarmVille': 1, 'BTIG': 1, 'Fox Interactive': 1, 'TOM': 1, 'Dow Jones': 1, 'the Wall Street Journal': 1, "Microsoft Corp's": 1, 'Yahoo Inc.': 1, 'Universal Music Group': 1, 'Warner Music Group': 1, 'EMI Music': 1, 'Myspace Music': 1, 'Paris Hilton': 1, 'Chernin

time: 1.83 s


In [9]:
counts = Counter([ent for ent, label in zip(items, labels) if label == 'ORG'])
matches = [
    [key, counts[key], securities_ds.get_most_similar_company(key)] for key in counts]
matches = [x for x in matches if x[2] is not None]
matched_stocks = dict([(x[0], x[2]) for x in matches])
matches

[["News Corp's", 8, 'News Corp.'],
 ['News Corp', 34, 'News Corp.'],
 ['Facebook', 24, 'Facebook'],
 ['AOL-Time Warner', 1, 'Time Warner Inc.'],
 ['Viacom', 4, 'Viacom Inc.'],
 ['News Corp.', 1, 'News Corp.'],
 ['Yahoo', 2, 'Yahoo Inc.'],
 ['Microsoft', 2, 'Microsoft Corp.'],
 ["Microsoft Corp's", 1, 'Microsoft Corp.'],
 ['Yahoo Inc.', 1, 'Yahoo Inc.'],
 ['Time Warner', 1, 'Time Warner Inc.'],
 ['Apple', 1, 'Apple Inc.']]

time: 1.18 s


In [67]:
securities_ds.get_most_similar_company('AOL-Time Warner', quiet=False)

                  Name  damerau_lev
441   Time Warner Inc.     0.266667
72          BorgWarner     0.600000
93              Cerner     0.666667
172  Estee Lauder Cos.     0.666667
5    Acuity Brands Inc     0.666667


'Time Warner Inc.'

time: 155 ms


In [66]:
found_entities = nlp_utils.find_nyse_corporations(reuters.loc[61727][2], quiet=False)

Found organisations: 234


time: 2.71 s


### Apply on Bloomberg Article

In [71]:
# idxmax bloomberg: 316777  (AAPL 1, FB 79, JPM 1, MSFT 1, MS 7)
bloomberg = pd.read_csv(BLOOMBERG, skiprows=316777, nrows=1, index_col=0)
# print(bloomberg.loc[316777][2])

time: 2.74 s


In [76]:
found_entities = nlp_utils.find_nyse_corporations(bloomberg.loc[316777][2], quiet=False)

Found ORG occurrences: 208
Organsations after filtering:
 - ORG (FB): 72
 - ORG (MS): 7
 - ORG (GS): 4
 - ORG (AAPL): 1
 - ORG (JPM): 1
 - ORG (MSFT): 1


time: 4.02 s
